In [1]:
import pandas as pd
from numpy import *
import matplotlib.pyplot as plt
import os, re, sys
top_dir = os.getcwd() 
print(top_dir)

C:\Users\aeberso2\OneDrive\Graduate School - MSU\Research - MSU\5d_things


In [2]:
# enthalpy information of each atom in the set
atoms_input = pd.read_excel('5d_experimental_vals.xlsx',sheetname='Atoms')
# experimental information for each compound in the set 
exp_vals= pd.read_excel('5d_experimental_vals.xlsx',sheetname='Cmpds_Mk2')
# computed dft energies for each compound in the set
dft_vals_ms = pd.read_excel('DFT_5d_enthal_mk_verify.xlsx',sheetname='Main')
dft_vals_so = pd.read_excel('DFT_5d_enthal_mk_verify.xlsx',sheetname='Main-SO')

In [3]:
HARTREE_to_KCAL = 627.509608
HARTREE_to_KJ  = 2625.5002

# needed to parse compound list for enthalpy calculation
# from the compound name, extracts a list of the constituent compounds
# then an array of how many times that element appears in the compound
# eg. WF6 returns ['W', 'F'] and [1, 6]
def create_atom_cmpd_list(usr_inp):
    int_set = []
    elem_list = []
    a = re.findall(r'[A-Z][a-z]*|\d+|\(|\)', usr_inp)
    for p in range(1,len(a)):
        if a[p-1].isalpha() and a[p].isalpha():
            if p == len(a)-1:
                int_set.append(1.0)
                int_set.append(1.0)
                elem_list.append(str(a[p-1]))
                elem_list.append(str(a[p]))
            else:
                int_set.append(1.0)
                elem_list.append(str(a[p-1]))               
                
        elif a[p-1].isalpha() and a[p].isdigit():
            int_set.append(float(a[p]))
            elem_list.append(str(a[p-1]))
    if a[-2].isdigit() and a[-1].isalpha():
        int_set.append(1)
        elem_list.append(str(a[-1]))     
        
    return array(elem_list), array(int_set)

# Color coating for Pandas tables in the jupyter notebook
# colors names are from the SVG standard
def enthal_color_quan(val):
    
    if val >= 0 and val < 6:
        color = 'steelblue'  
    elif (val) < 0 and (val) > -6:
        color = 'steelblue'   
    elif val >= 6 and val < 20:
        color = 'darkgoldenrod'  
    elif (val) <= -6 and (val) > -20:
        color = 'olive'           
    elif val >= 20 and val < 50:
        color = 'orangered'  
    elif (val) <= -20 and (val) > -50:
        color = 'chocolate'   
    elif val >= 50 and val < 400:
        color = 'crimson'  
    elif (val) <= -50 and (val) > -400:
        color = 'firebrick'             
    else:
        color = 'k'
    return 'color: {}'.format(color)

def dft_energy_error(atoms_input,exp_vals,dft_vals,check_consistency):
    # currently uses the JANAF experimental values, it takes all non blank entries 
    Janaf_exp_vals = exp_vals.loc['JANAF_exp','Au2':][exp_vals.loc['JANAF_exp','Au2':].notnull()]
    # a list of the calculated ZPE + H_inc
    Thermo_info = atoms_input.loc['H-H0':'DeltaHf298',:]
    # list of functional values for atoms then compounds 
    dft_atms = dft_vals.loc['SVWN':'B2PLYP','Hf':'I']
    dft_cmpds = dft_vals.loc['SVWN':'B2PLYP','Au2':]

    # temp array to hold values
    TEMP_HOLD = []
    # for each compound in the compound index
    for compd in dft_cmpds.axes[1][:]:
        # create the atom set adn integer list
        atom_set, int_list = create_atom_cmpd_list(compd)
        # start the sum for the first index multiply the number of times that atom appears in the cmpound
        # times the dft calculated atomic energy + the \ H_{inc} as (H(298) - H(0))
        # the minus 1 corrects the fact that Janaf lists the values as H(0) - H(298)
        sum_Eatom_Hinc = 0
        for i in range(0,len(atom_set)):
            sum_Eatom_Hinc += int_list[i]*(dft_atms.loc[:,atom_set[i]] 
                                           + (-1*Thermo_info.loc['H-H0', atom_set[i]]/HARTREE_to_KCAL)
                                          )

        # Add the calculated ZPVE and vibrational contributitons to enthalpy 
        Delta_H_sum = 0        
        for i in range(0,len(atom_set)):
            Delta_H_sum += (int_list[i]*Thermo_info.loc['DeltaHf298', atom_set[i]])/HARTREE_to_KCAL
        # compute total enthalpy 
        Total_enthalpy = (dft_cmpds.loc[:,compd] 
                          + dft_vals.loc['ThermC (Enthalpy)', compd] 
                          - sum_Eatom_Hinc 
                          + Delta_H_sum)*HARTREE_to_KCAL
        
        TEMP_HOLD.append(array(Total_enthalpy))

    Vals = pd.DataFrame(array(TEMP_HOLD), index=dft_cmpds.axes[1][:], columns=dft_cmpds.axes[0])

    # remove the elements for which there are no Janaf exp vals
    comparison_data = Vals.loc[Janaf_exp_vals.axes[0],:]
    signed_errors = comparison_data.copy()
    # compute difference from experiment, i.e. the signed error
    for compound in Janaf_exp_vals.axes[0]:
        signed_errors.loc[compound,:] = comparison_data.loc[compound,:] - Janaf_exp_vals[compound ] 
    # remove rows where all functionals have NaN values 
    # TPSS is chosen since optimizations were done with tpss, so if it has no value 
    # then no other functionals will have a value
    current_energy_vals = comparison_data[comparison_data.loc[:,'TPSS'].notnull()]
    current_error_vals = signed_errors[signed_errors.loc[:,'TPSS'].notnull()]

    # for determining problem molecules were functions give vastly different values amongst themselves
    if check_consistency == 'Yes':
        std_set = []
        for i in current_energy_vals.axes[0]:
            std_set.append(current_energy_vals.loc[i,:].std())
        consti_check = pd.Series(std_set,index=current_energy_vals.axes[0])

        state_is_consistent = consti_check[consti_check < 35]

        state_is_not_consistent = consti_check[consti_check >= 35]

        consistent_error_set = current_error_vals.loc[state_is_consistent.axes[0],:]

        inconsistent_error_set = current_error_vals.loc[state_is_not_consistent.axes[0],:]

        return current_error_vals, consistent_error_set, inconsistent_error_set, current_energy_vals
    else:
        return current_error_vals, current_energy_vals



In [4]:
# ,initial_vals_ms, initial_vals_so
error_recp, cons_error, incons_error, energy_recp = dft_energy_error(atoms_input,exp_vals,dft_vals_ms,'Yes')
error_so, energy_so = dft_energy_error(atoms_input,exp_vals,dft_vals_so,'No')

In [11]:
# energy_recp
# energies without SO correction 

In [7]:
error_recp.style.applymap(enthal_color_quan)

,SVWN,BP86,BLYP,PW91,PBE,TPSS,M06-L,B3P86,X3LYP,B97-1,B3LYP,PBE0,MPW1K,BHLYP,TPSSH,M06,M06-2X,M11,B2PLYP
Au2,-12.8932,1.66489,5.6993,0.731013,1.2408,1.34554,1.06134,5.27196,8.58824,4.28881,8.91035,7.12365,11.6706,15.227,3.74503,4.53017,18.8411,8.43647,4.99372
AuH,-10.4375,0.265653,4.18034,3.20908,3.60261,2.11467,7.42679,3.33554,6.79052,6.86296,6.60672,8.30312,11.5805,12.5977,3.76711,10.3484,13.5533,nan,nan
AuF3,-30.6145,29.9959,36.8493,24.9294,26.811,34.8658,50.1009,51.4022,58.7178,53.9535,58.7623,61.6685,86.6051,96.8192,48.3111,66.1062,77.0488,57.1862,59.9039
AuCl,40.0402,56.375,61.4281,54.9242,55.1763,56.2912,54.1527,58.2438,62.6044,57.4267,62.975,59.1913,62.3018,67.1599,57.8772,58.0771,nan,56.7677,61.418
AuCl3,11.572,61.1536,77.0616,56.2736,57.3932,62.6547,64.2939,72.373,85.5937,75.0057,86.593,76.171,91.2318,107.506,69.7705,79.9965,96.9359,89.5808,83.834
Au2Cl2,183.494,88.5497,265.321,240.201,241.639,49.7576,201.31,88.2751,107.313,93.5381,109.552,297.517,95.2456,114.847,90.1172,225.197,103.226,91.6223,nan
Au2Cl4,15.1275,106.966,140.367,97.4575,99.5848,105.163,204.003,115.077,141.997,123.673,144.975,117.799,134.32,164.644,150.829,135.138,147.501,131.615,nan
AuBr,39.8162,54.8087,59.4093,53.3997,53.8361,54.2483,57.1841,56.5875,60.4791,56.3689,60.8776,57.4697,60.1911,64.5785,55.7177,60.236,63.354,53.9778,59.1248
AuBr3,2.31278,53.656,68.9158,48.751,50.3953,52.9202,65.4549,57.6542,70.0332,61.456,71.22,60.8838,73.7584,88.6344,54.3884,72.1931,87.6968,70.282,68.8698
HfO,-25.3565,1.60195,3.24421,0.65696,1.17029,11.8828,-8.49795,15.731,18.3178,16.3877,17.6997,24.6996,31.3298,45.758,19.5064,15.1807,30.0033,25.2117,10.1346


In [8]:
mCmpds = ['HfCl4','Au2','WCl4','HgCl2','TaCl5','WF6']

Mfunc = ['SVWN', 'BP86', 'BLYP', 'PW91', 'PBE', 'TPSS', 'M06-L', 'B3P86',
       'X3LYP', 'B97-1', 'B3LYP', 'PBE0', 'BHLYP', 'TPSSH']
error_recp.loc[mCmpds,Mfunc].style.applymap(enthal_color_quan)

,SVWN,BP86,BLYP,PW91,PBE,TPSS,M06-L,B3P86,X3LYP,B97-1,B3LYP,PBE0,BHLYP,TPSSH
HfCl4,-72.0599,-12.1557,7.0228,-20.078,-18.1624,-16.2585,-29.1735,-14.1169,3.24935,-12.0796,4.84129,-9.34328,17.1263,-13.4961
Au2,-12.8932,1.66489,5.6993,0.731013,1.2408,1.34554,1.06134,5.27196,8.58824,4.28881,8.91035,7.12365,15.227,3.74503
WCl4,-55.9376,3.81613,19.4534,-2.21543,-1.01302,-7.78909,-12.8171,12.126,24.9326,11.3711,27.2656,16.9862,47.0915,10.3878
HgCl2,-22.9589,3.54315,13.0594,0.381912,0.801366,-0.526557,-4.75885,3.12583,11.0726,4.04228,12.0335,3.17129,14.4574,3.07637
TaCl5,-55.7335,15.0313,35.4189,6.52454,8.59375,-23.6686,-37.2111,30.72,48.7007,31.0562,49.6605,36.9736,81.3384,25.9516
WF6,57.0997,156.157,153.263,148.318,151.085,-56.5653,-32.3543,240.71,242.897,240.248,240.022,271.392,370.369,227.197


In [9]:
error_so.loc[mCmpds,Mfunc].style.applymap(enthal_color_quan)

,SVWN,BP86,BLYP,PW91,PBE,TPSS,M06-L,B3P86,X3LYP,B97-1,B3LYP,PBE0,BHLYP,TPSSH
HfCl4,nan,nan,nan,nan,nan,-15.922,-53.2474,nan,nan,nan,nan,nan,nan,nan
Au2,nan,nan,nan,nan,nan,-2.25009,nan,nan,nan,nan,nan,nan,nan,nan
WCl4,nan,nan,nan,nan,nan,-8.95262,-13.2811,nan,nan,nan,nan,nan,nan,nan
HgCl2,nan,nan,nan,nan,nan,-2.06154,-6.22882,nan,nan,nan,nan,nan,nan,nan
TaCl5,nan,nan,nan,nan,nan,-22.3456,-35.7267,nan,nan,nan,nan,nan,nan,nan
WF6,nan,nan,nan,nan,nan,-55.0097,-30.025,nan,nan,nan,nan,nan,nan,nan


### Uncomment block below to print to LaTeX output

In [10]:
# tex_file = open('5d_differences.tex','w')
# print(first_half.round(2).to_latex(),file=tex_file)
# print(second_half.round(2).to_latex(),file=tex_file)
# tex_file.close()
# print(error_recp.T.round(2).to_latex())
# print(error_so.loc[mCmpds,Mfunc].T.round(2).to_latex())
